# Exercise 1

Welcome to the first challenge! The goal of this exercise is to create a Public Transport Agent that is able to access public transport information to answer questions regarding connections. Part of the code is already provided for you, but you will find TODO comments that indicate where you should implement something. Good luck with the exercise!

## Setup
Let's start by importing the libraries that we will need.

In [ ]:
import warnings

warnings.filterwarnings("ignore")

In [ ]:
import os
import re
from datetime import datetime

import requests
from crewai import Agent, Crew, Task
from crewai_tools import BaseTool, tool
from dotenv import load_dotenv

Before you execute this cell, make sure to provide the environment variables `OPENAI_API_BASE`, `OPENAI_MODEL_NAME`, and `OPENAI_API_KEY` in the `.env` file.

In [ ]:
load_dotenv(override=True)

assert "OPENAI_MODEL_NAME" in os.environ, "No model specified in .env file!"
print("Using the following LLM model:", os.environ.get("OPENAI_MODEL_NAME"))

## Helper functions

In this section of the notebook you can implement the helper functions that your tools are going to use. There is already one helper function that is implemented for you. You may or may not use it. If you don't need any other helper function, feel free to just execute the cell without modyfing it.

In [ ]:
# Helper functions
def format_duration(duration: str) -> str:
    """ Formats the duration returned by the public transport API such that it is more readable. """
    readable_duration_str = "Invalid format"
    
    match = re.match(r"(\d{2})d(\d{2}):(\d{2}):(\d{2})", duration)
    if match:
        days, hours, minutes, seconds = map(int, match.groups())
        
        readable_duration = []
        
        if days > 0:
            readable_duration.append(f"{days} day{'s' if days > 1 else ''}")
        if hours > 0:
            readable_duration.append(f"{hours} hour{'s' if hours > 1 else ''}")
        if minutes > 0:
            readable_duration.append(f"{minutes} minute{'s' if minutes > 1 else ''}")
        if seconds > 0:
            readable_duration.append(f"{seconds} second{'s' if seconds > 1 else ''}")
        
        readable_duration_str = ', '.join(readable_duration)

    return readable_duration_str

## Tools

In this section, we will implement the tools that our agent is going to use. You can turn any function into a tool by using the `@tool` decorator. The decorator will turn the function into a tool object of type `Tool`, which in turn is of type `BaseTool`. By default, these `BaseTools` come with a caching mechanism. If you don't want that, I'd recommend writing the tool as a class that inherits from `BaseTool` and then explicitly setting the `cache_function` to a function that returns `False`. Below you can see both ways of creating a tool.

Now it's your turn to have a look at the TODO comments below and implement the missing parts. Feel free to create more cells to play around with the implementation and to test it.

In [ ]:
# Tools
@tool("Get public transport connections")
def get_connections(start: str, end: str, date: str, time: str, is_arrival_time: bool):
    """
    Gets public transport connections for a given start and end location and a specific date and time.
    Requires UTF-8 encoded arguments, do not use unicode characters! 

    :param start: A string representing either the name of the station or its ID
    :param end: A string representing either the name of the station or its ID
    :param date: The date for which to check the connections (iso format)
    :param time: The time for which to check the connections (%H:%M)
    :param is_arrival_time: Boolean value specifying whether the date and time refer 
        to the arrival (True) or the departure (False). The argument should be formatted
        as a string because it will be converted into a boolean upon executing this tool.
    """
    # TODO: You might want to experiment with the following two lines in another cell first to get a grasp of how the response looks like
    response = requests.get(f"http://transport.opendata.ch/v1/connections?from={start}&to={end}&date={date}&time={time}&isArrivalTime={int(is_arrival_time)}")
    data = response.json()

    connections = []
    for connection in data["connections"]:
        connections.append(
            {
                # TODO: Think about what kind of information you want to provide for the agent and then put it in the dictionary
            }
        )

    if not connections:
        raise Exception("Couldn't find any connection, please verify that all of the arguments are correctly formatted in UTF-8")

    return connections

class GetCurrentDateAndTimeTool(BaseTool):
    name: str = "Get the current date and time"
    description: str = "Returns a dictionary with information about the current date and time."

    def _run(self) -> dict:
        # TODO: implement this tool
        pass

    def cache_function(*args):
        return False

## Agent

Now we need to create an agent that can call the tools that we just created. Below you can see the template for the public transport agent. You need to specify the rolem the goal and the backstory of the agent. Usually, the more specific you are, the better the outcome. When defining the role of an agent, it can be helpful to think about it from the perspective of a project manager that needs to find the perfect candidate for a certain position. What kind of roles would he be hiring for?

You can use curly brackets within the text fields to represent variables that are inserted when kicking off the crew. This way, your descriptions are more flexible.

In [ ]:
public_transport_agent = Agent(
    role="",  # TODO: Specify the role of the agent
    goal="",  # TODO: Specify the goal of the agent, you might want to include the variable user_request
    backstory="",  # TODO: Specify the backstory of the agent
    tools=[GetCurrentDateAndTimeTool(), get_connections],
    verbose=True,
)

## Task
Now that we have an agent that can use tools, we need to define the task that should be solved. A task consists of a description, an expected output, and an agent that is assigned to the task. Again, you can use curly brackets to represent variables that are inserted at runtime.

In [ ]:
public_transport_task = Task(
    description="",  # TODO: Specify the description of the task
    expected_output="",  # TODO: Specify the expected output of the task
    agent=public_transport_agent,
)

## Kicking off the Crew

Now it's time to kick off the crew. This part is already implemented for you, so go ahead an execute it. You will be asked for a question, here are some inspirations:

- Ich befinde mich gerade beim Zürich HB und möchte nach Bern fahren, wann ist die nächste Verbindung und auf welchem Gleis?
- Ich befinde mich in Bern. Wann muss ich morgen auf den Zug gehen, wenn ich um 11 Uhr in Zürich sein will?
- Ich bin etwa 15 Minuten Fussweg vom Bahnof Luzern entfernt. Wann muss ich von hier loslaufen, wenn ich um 17 Uhr in Basel sein will?

In [ ]:
crew = Crew(agents=[public_transport_agent], tasks=[public_transport_task])

question = input("Question: ")

result = crew.kickoff(inputs={
    "user_request": question
})

print("Reply:", result.raw)

In [ ]:
print(f"Usage metrics: {crew.usage_metrics}")